# Database Build for Advanced Problems - Main Tables (4 Tables)
### Notebook to Create SQL Database for SQL Tutorial


### Import libraries
NOTE: Used magic command of %%bigquery to execute sql and save to pandas dataframe

In [27]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import bigquery
pd.set_option('display.max_columns', None)

%load_ext google.cloud.bigquery
# for large datasets store data in google storage for improved speed using the following command
# %bigquery --project job-listings-366015 --use_bqstorage_api 

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [28]:
%%bigquery gsearch_jobs_wide
-- filter search_time to 2023-01-01 to 2023-12-31
SELECT *
FROM `job-listings-366015.gsearch_job_listings_clean.gsearch_jobs_wide`
WHERE search_time BETWEEN '2023-01-01' AND '2024-01-01'


Query is running:   0%|          |

Downloading:   0%|          |

## Drop Duplicates

In [29]:
# print("Dataset size (with Duplicates):", len(gsearch_jobs_wide))

# # Specify the columns to watch
# # columns_to_consider = ['job_title_final', 'job_title_clean', 'job_title', 'company_name', 'search_country', 'job_location', 'job_via', 'job_salary', 'job_schedule_type', 'job_work_from_home', 'job_commute_time', 'job_extensions', 'error', 'search_term', 'search_location', 'search_time', 'job_no_degree_mention', 'job_health_insurance', 'salary_pay', 'salary_rate', 'salary_avg', 'salary_min', 'salary_max', 'salary_year', 'salary_hour']
# columns_to_consider = ['job_title', 'job_description'] #, 'job_salary']

# # Remove duplicates considering only the columns_to_consider
# gsearch_jobs_wide = gsearch_jobs_wide.drop_duplicates(subset=columns_to_consider)

# print("Dataset size (withOut Duplicates):", len(gsearch_jobs_wide))

# # analyze the search_time column, displaying it as a histogram
# gsearch_jobs_wide['search_time'].hist(bins=100)

## Drop Unnecessary Columns

In [30]:
gsearch_jobs_wide.head()

columns_to_drop = ["job_title_clean", "job_id", "job_posted_at", "search_term", "search_id", "salary_min", "salary_max", "salary_avg"]

# drop columns
job_postings_fact = gsearch_jobs_wide.drop(columns=columns_to_drop)

# create a job_id column
job_postings_fact["job_id"] = job_postings_fact.index

# reorder columns with job_id first
job_postings_fact = job_postings_fact.set_index('job_id').reset_index()

job_postings_fact.head()

,job_id,job_title_final,job_title,company_name,job_location,job_via,job_description,job_highlights_qualifications,job_highlights_responsibilities,job_highlights_benefits,job_salary,job_schedule_type,job_work_from_home,job_commute_time,job_extensions,company_link,company_link_google,company_thumbnail,error,search_location,search_time,job_no_degree_mention,job_health_insurance,keywords_programming,keywords_databases,keywords_cloud,keywords_libraries,keywords_webframeworks,keywords_os,keywords_analyst_tools,keywords_other,keywords_async,keywords_sync,keywords_all,search_country,salary_pay,salary_rate,salary_year,salary_hour
0,0,Data Scientist,Principal Data Scientist,Quest Global Services Pte. Ltd.,Singapore,via Recruit.net,A Principal Data Scientist will lead data engi...,None,None,None,None,Full-time,<NA>,None,"[6 hours ago, SGD 96K–SGD 144K a month, Full-t...",http://www.quest-global.com/,https://www.google.com/search?ucbcb=1&hl=en&gl...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,Singapore,2023-03-11 06:18:19,<NA>,<NA>,"{'list': [{'element': 'scala'}, {'element': 's...",None,None,None,None,None,None,None,None,None,"{'list': [{'element': 'scala'}, {'element': 's...",Singapore,None,None,NaN,NaN
1,1,Software Engineer,Software System Engineer,ALSTOM,Sweden,via WANE Jobs,"Date: 16-Feb-2023\n\nLocation: Vasteras, SE\n\...",None,None,None,None,Full-time,<NA>,None,"[13 hours ago, Full-time]",None,https://www.google.com/search?q=ALSTOM&sa=X&ve...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,Sweden,2023-04-07 06:12:23,<NA>,<NA>,None,None,None,None,None,None,None,None,None,None,None,Sweden,None,None,NaN,NaN
2,2,Senior Data Analyst,SCADA Analyst - Expert Level,Dice,"Midland, TX",via LinkedIn,Dice is the leading career destination for tec...,"[""Must work well with field I&E employees and ...","[""Activities include all aspects of support fr...",None,None,Full-time,<NA>,None,"[15 hours ago, Full-time, No degree mentioned]",None,https://www.google.com/search?ucbcb=1&hl=en&gl...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,"Texas, United States",2023-02-28 06:01:12,True,<NA>,{'list': [{'element': 'sql'}]},None,None,None,None,None,None,None,None,None,{'list': [{'element': 'sql'}]},United States,None,None,NaN,NaN
3,3,Data Analyst,Data Analyst,Lesta Games,"Minsk, Belarus",via hh.ru,Lesta Games ведёт разработку игровых проектов ...,None,None,None,None,Full-time,<NA>,None,"[20 hours ago, Full-time, No degree mentioned]",https://lesta.ru/ru,https://www.google.com/search?gl=us&hl=en&q=Le...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,Belarus,2023-04-08 06:51:33,True,<NA>,"{'list': [{'element': 'sql'}, {'element': 'pyt...",None,None,"{'list': [{'element': 'pandas'}, {'element': '...",None,None,None,None,None,None,"{'list': [{'element': 'sql'}, {'element': 'pyt...",Belarus,None,None,NaN,NaN
4,4,Data Engineer,"Data Engineer, Analytics",Shutterstock,"Dublin, Ireland",via Trabajo.org,"Shutterstock is growing by leaps and bounds, a...",None,None,None,None,Full-time,<NA>,None,"[19 hours ago, Full-time]",http://www.shutterstock.com/,https://www.google.com/search?hl=en&gl=us&q=Sh...,None,False,Ireland,2023-04-29 06:12:17,<NA>,<NA>,"{'list': [{'element': 'sql'}, {'element': 'pyt...",{'list': [{'element': 'dynamodb'}]},"{'list': [{'element': 'aws'}, {'element': 'sno...",None,None,None,{'list': [{'element': 'looker'}]},None,None,None,"{'list': [{'element': 'looker'}, {'element': '...",Ireland,None,None,NaN,NaN


## Cleanup Job Table

In [31]:
columns_renamed = {
    "search_country": "job_country",
    "salary_year": "salary_year_avg",
    "salary_hour": "salary_hour_avg",
    "search_time": "job_posted_date"
}

# rename columns of job_posting_fact
job_postings_fact = job_postings_fact.rename(columns=columns_renamed)

# convert job_posted_date to a date only object vice date_time
job_postings_fact['job_posted_date'] = job_postings_fact['job_posted_date'].dt.date

# convert job_posted_date to a datetime (needed to upload to bigquery)
job_postings_fact['job_posted_date'] = pd.to_datetime(job_postings_fact['job_posted_date'])


In [32]:
job_postings_fact.head()

,job_id,job_title_final,job_title,company_name,job_location,job_via,job_description,job_highlights_qualifications,job_highlights_responsibilities,job_highlights_benefits,job_salary,job_schedule_type,job_work_from_home,job_commute_time,job_extensions,company_link,company_link_google,company_thumbnail,error,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,keywords_programming,keywords_databases,keywords_cloud,keywords_libraries,keywords_webframeworks,keywords_os,keywords_analyst_tools,keywords_other,keywords_async,keywords_sync,keywords_all,job_country,salary_pay,salary_rate,salary_year_avg,salary_hour_avg
0,0,Data Scientist,Principal Data Scientist,Quest Global Services Pte. Ltd.,Singapore,via Recruit.net,A Principal Data Scientist will lead data engi...,None,None,None,None,Full-time,<NA>,None,"[6 hours ago, SGD 96K–SGD 144K a month, Full-t...",http://www.quest-global.com/,https://www.google.com/search?ucbcb=1&hl=en&gl...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,Singapore,2023-03-11,<NA>,<NA>,"{'list': [{'element': 'scala'}, {'element': 's...",None,None,None,None,None,None,None,None,None,"{'list': [{'element': 'scala'}, {'element': 's...",Singapore,None,None,NaN,NaN
1,1,Software Engineer,Software System Engineer,ALSTOM,Sweden,via WANE Jobs,"Date: 16-Feb-2023\n\nLocation: Vasteras, SE\n\...",None,None,None,None,Full-time,<NA>,None,"[13 hours ago, Full-time]",None,https://www.google.com/search?q=ALSTOM&sa=X&ve...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,Sweden,2023-04-07,<NA>,<NA>,None,None,None,None,None,None,None,None,None,None,None,Sweden,None,None,NaN,NaN
2,2,Senior Data Analyst,SCADA Analyst - Expert Level,Dice,"Midland, TX",via LinkedIn,Dice is the leading career destination for tec...,"[""Must work well with field I&E employees and ...","[""Activities include all aspects of support fr...",None,None,Full-time,<NA>,None,"[15 hours ago, Full-time, No degree mentioned]",None,https://www.google.com/search?ucbcb=1&hl=en&gl...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,"Texas, United States",2023-02-28,True,<NA>,{'list': [{'element': 'sql'}]},None,None,None,None,None,None,None,None,None,{'list': [{'element': 'sql'}]},United States,None,None,NaN,NaN
3,3,Data Analyst,Data Analyst,Lesta Games,"Minsk, Belarus",via hh.ru,Lesta Games ведёт разработку игровых проектов ...,None,None,None,None,Full-time,<NA>,None,"[20 hours ago, Full-time, No degree mentioned]",https://lesta.ru/ru,https://www.google.com/search?gl=us&hl=en&q=Le...,https://encrypted-tbn0.gstatic.com/images?q=tb...,False,Belarus,2023-04-08,True,<NA>,"{'list': [{'element': 'sql'}, {'element': 'pyt...",None,None,"{'list': [{'element': 'pandas'}, {'element': '...",None,None,None,None,None,None,"{'list': [{'element': 'sql'}, {'element': 'pyt...",Belarus,None,None,NaN,NaN
4,4,Data Engineer,"Data Engineer, Analytics",Shutterstock,"Dublin, Ireland",via Trabajo.org,"Shutterstock is growing by leaps and bounds, a...",None,None,None,None,Full-time,<NA>,None,"[19 hours ago, Full-time]",http://www.shutterstock.com/,https://www.google.com/search?hl=en&gl=us&q=Sh...,None,False,Ireland,2023-04-29,<NA>,<NA>,"{'list': [{'element': 'sql'}, {'element': 'pyt...",{'list': [{'element': 'dynamodb'}]},"{'list': [{'element': 'aws'}, {'element': 'sno...",None,None,None,{'list': [{'element': 'looker'}]},None,None,None,"{'list': [{'element': 'looker'}, {'element': '...",Ireland,None,None,NaN,NaN


## Create Company Table

In [33]:
company_dim = job_postings_fact

# keep only columns that start with company
company_dim = company_dim.filter(regex='^company', axis=1)

# save column names into a list
col_names_company = company_dim.columns.tolist()

# drop duplicates based on company_name
company_dim = company_dim.drop_duplicates(subset=['company_name'])

# remove company_ from beginning of column names
company_dim.columns = company_dim.columns.str.replace('company_', '')

# create company_id column  and set as index
company_dim['company_id'] = company_dim.index

# make company_id the first column
company_dim = company_dim.set_index('company_id').reset_index()

# company_dim.to_csv('SQL_Database_Build/company_dim.csv', index=False)

company_dim.head()

,company_id,name,link,link_google,thumbnail
0,0,Quest Global Services Pte. Ltd.,http://www.quest-global.com/,https://www.google.com/search?ucbcb=1&hl=en&gl...,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,1,ALSTOM,None,https://www.google.com/search?q=ALSTOM&sa=X&ve...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,2,Dice,None,https://www.google.com/search?ucbcb=1&hl=en&gl...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,3,Lesta Games,https://lesta.ru/ru,https://www.google.com/search?gl=us&hl=en&q=Le...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,4,Shutterstock,http://www.shutterstock.com/,https://www.google.com/search?hl=en&gl=us&q=Sh...,None


## Cleanup Company Info in fact table


In [34]:
# merge on company_id only on to job_postings_fact from company_dim
job_postings_fact = pd.merge(job_postings_fact, company_dim[['company_id', 'name']],left_on='company_name', right_on='name', how='left')

# drop column names that start with 'name' or are in the list col_names_company
job_postings_fact = job_postings_fact.drop(columns=col_names_company)
job_postings_fact = job_postings_fact.drop(columns=['name'])

job_postings_fact.head()

,job_id,job_title_final,job_title,job_location,job_via,job_description,job_highlights_qualifications,job_highlights_responsibilities,job_highlights_benefits,job_salary,job_schedule_type,job_work_from_home,job_commute_time,job_extensions,error,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,keywords_programming,keywords_databases,keywords_cloud,keywords_libraries,keywords_webframeworks,keywords_os,keywords_analyst_tools,keywords_other,keywords_async,keywords_sync,keywords_all,job_country,salary_pay,salary_rate,salary_year_avg,salary_hour_avg,company_id
0,0,Data Scientist,Principal Data Scientist,Singapore,via Recruit.net,A Principal Data Scientist will lead data engi...,None,None,None,None,Full-time,<NA>,None,"[6 hours ago, SGD 96K–SGD 144K a month, Full-t...",False,Singapore,2023-03-11,<NA>,<NA>,"{'list': [{'element': 'scala'}, {'element': 's...",None,None,None,None,None,None,None,None,None,"{'list': [{'element': 'scala'}, {'element': 's...",Singapore,None,None,NaN,NaN,0
1,1,Software Engineer,Software System Engineer,Sweden,via WANE Jobs,"Date: 16-Feb-2023\n\nLocation: Vasteras, SE\n\...",None,None,None,None,Full-time,<NA>,None,"[13 hours ago, Full-time]",False,Sweden,2023-04-07,<NA>,<NA>,None,None,None,None,None,None,None,None,None,None,None,Sweden,None,None,NaN,NaN,1
2,2,Senior Data Analyst,SCADA Analyst - Expert Level,"Midland, TX",via LinkedIn,Dice is the leading career destination for tec...,"[""Must work well with field I&E employees and ...","[""Activities include all aspects of support fr...",None,None,Full-time,<NA>,None,"[15 hours ago, Full-time, No degree mentioned]",False,"Texas, United States",2023-02-28,True,<NA>,{'list': [{'element': 'sql'}]},None,None,None,None,None,None,None,None,None,{'list': [{'element': 'sql'}]},United States,None,None,NaN,NaN,2
3,3,Data Analyst,Data Analyst,"Minsk, Belarus",via hh.ru,Lesta Games ведёт разработку игровых проектов ...,None,None,None,None,Full-time,<NA>,None,"[20 hours ago, Full-time, No degree mentioned]",False,Belarus,2023-04-08,True,<NA>,"{'list': [{'element': 'sql'}, {'element': 'pyt...",None,None,"{'list': [{'element': 'pandas'}, {'element': '...",None,None,None,None,None,None,"{'list': [{'element': 'sql'}, {'element': 'pyt...",Belarus,None,None,NaN,NaN,3
4,4,Data Engineer,"Data Engineer, Analytics","Dublin, Ireland",via Trabajo.org,"Shutterstock is growing by leaps and bounds, a...",None,None,None,None,Full-time,<NA>,None,"[19 hours ago, Full-time]",False,Ireland,2023-04-29,<NA>,<NA>,"{'list': [{'element': 'sql'}, {'element': 'pyt...",{'list': [{'element': 'dynamodb'}]},"{'list': [{'element': 'aws'}, {'element': 'sno...",None,None,None,{'list': [{'element': 'looker'}]},None,None,None,"{'list': [{'element': 'looker'}, {'element': '...",Ireland,None,None,NaN,NaN,4


## Skills to Job Table

In [35]:
skills_job_dim = job_postings_fact

# keep only columns that start with 'keyords_' and 'job_id'
skills_job_dim = skills_job_dim.filter(regex='^keywords_|job_id', axis=1)

#remove keywords_ from beginning of column names
skills_job_dim.columns = skills_job_dim.columns.str.replace('keywords_', '')

# drop the 'all' column
skills_job_dim = skills_job_dim.drop(columns=['all'])

# unpivot the dataframe keeping the job_id column
skills_job_dim = skills_job_dim.melt(id_vars=['job_id'], var_name='type', value_name='skills')

# drop None values in skills column
skills_job_dim = skills_job_dim.dropna(subset=['skills'])

# First, transform the 'skills' column into a list of elements
skills_job_dim['skills'] = skills_job_dim['skills'].apply(lambda x: [item['element'] for item in x['list']])

# Then, use explode to create a new row for each element in the lists
skills_job_dim = skills_job_dim.explode('skills')

skills_job_dim

,job_id,type,skills
0,0,programming,scala
0,0,programming,sql
0,0,programming,python
2,2,programming,sql
3,3,programming,sql
...,...,...,...
18266667,1826511,sync,unify
18266672,1826516,sync,twilio
18266759,1826603,sync,twilio
18266760,1826604,sync,unify


## Skill Table

In [36]:
skills_dim = skills_job_dim[['skills', 'type']].drop_duplicates().reset_index(drop=True).reset_index()
skills_dim.columns = ['skill_id', 'skills', 'type']

# skills_dim.to_csv('SQL_Database_Build/skills_dim.csv', index=False)

skills_dim

,skill_id,skills,type
0,0,scala,programming
1,1,sql,programming
2,2,python,programming
3,3,r,programming
4,4,shell,programming
...,...,...,...
254,254,ringcentral,sync
255,255,webex,sync
256,256,symphony,sync
257,257,mattermost,sync


## Skill to Job Table - Add Skill ID and remove skill

In [37]:
skills_job_dim = pd.merge(skills_job_dim, skills_dim[['skill_id', 'skills']], on=['skills'], how='left')

skills_job_dim = skills_job_dim[['job_id', 'skill_id']]

# skills_job_dim.to_csv('SQL_Database_Build/skills_job_dim.csv', index=False)

skills_job_dim

,job_id,skill_id
0,0,0
1,0,1
2,0,2
3,2,1
4,3,1
...,...,...
10200364,1826511,247
10200365,1826516,253
10200366,1826603,253
10200367,1826604,247


In [38]:
job_postings_fact = job_postings_fact.drop(columns=['job_description', 'job_highlights_qualifications', 'job_highlights_responsibilities', 'job_highlights_benefits' , 'job_salary', 'job_commute_time', 'job_extensions', 'error', 'salary_pay'])

#remove columns that start with "keywords_"
job_postings_fact = job_postings_fact.filter(regex='^(?!keywords_)', axis=1)

# rename column job_title_final to job_title_simple
job_postings_fact = job_postings_fact.rename(columns={'job_title_final': 'job_title_short'})

# replace '<NA>' values with False in job_work_from_home, job_no_degree_mention, and job_health_insurance columns
job_postings_fact['job_work_from_home'] = job_postings_fact['job_work_from_home'].fillna(False)
job_postings_fact['job_no_degree_mention'] = job_postings_fact['job_no_degree_mention'].fillna(False)
job_postings_fact['job_health_insurance'] = job_postings_fact['job_health_insurance'].fillna(False)

#remove unsual spaces before and after values in column
job_postings_fact['job_location'] = job_postings_fact['job_location'].str.strip()

# move company_id column to the right after job_id
cols = list(job_postings_fact.columns.values)
cols.pop(cols.index('company_id'))
job_postings_fact = job_postings_fact[['company_id']+ cols]

# set job_id as index
job_postings_fact = job_postings_fact.set_index('job_id').reset_index()

job_postings_fact

# job_postings_fact.to_csv('SQL_Database_Build/job_postings_fact.csv', index=False)

,job_id,company_id,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg
0,0,0,Data Scientist,Principal Data Scientist,Singapore,via Recruit.net,Full-time,False,Singapore,2023-03-11,False,False,Singapore,None,NaN,NaN
1,1,1,Software Engineer,Software System Engineer,Sweden,via WANE Jobs,Full-time,False,Sweden,2023-04-07,False,False,Sweden,None,NaN,NaN
2,2,2,Senior Data Analyst,SCADA Analyst - Expert Level,"Midland, TX",via LinkedIn,Full-time,False,"Texas, United States",2023-02-28,True,False,United States,None,NaN,NaN
3,3,3,Data Analyst,Data Analyst,"Minsk, Belarus",via hh.ru,Full-time,False,Belarus,2023-04-08,True,False,Belarus,None,NaN,NaN
4,4,4,Data Engineer,"Data Engineer, Analytics","Dublin, Ireland",via Trabajo.org,Full-time,False,Ireland,2023-04-29,False,False,Ireland,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826679,1826679,4650,Machine Learning Engineer,Machine Learning Engineer Pleno,"Brasília - Brasilia, Federal District, Brazil",via Empregos Trabajo.org,Full-time,False,Brazil,2023-11-14,False,False,Brazil,None,NaN,NaN
1826680,1826680,68785,Data Analyst,data analyst.,Argentina,via LinkedIn,Full-time,False,Argentina,2023-11-04,True,False,Argentina,None,NaN,NaN
1826681,1826681,65,Data Scientist,Staff Data Scientist Solutions Design Team!,"Fayetteville, AR",via ZipRecruiter,Full-time,False,Sudan,2023-09-13,False,False,Sudan,None,NaN,NaN
1826682,1826682,36,Senior Data Engineer,"Senior Data Engineer - Hybrid- Sacramento, CA","Sacramento, CA",via ZipRecruiter,Full-time,False,"New York, United States",2023-07-20,False,True,United States,None,NaN,NaN


## Cleanup Unsual Line Terminators

In [39]:
# Remove unusual line terminators from dataframes as causing issues with PostgreSQL import 

line_terminators = ['\r', '\r\n', '\n', r'\\u\w{4}']
replacer = ''
job_postings_fact = job_postings_fact.replace(line_terminators, replacer, regex=True)
company_dim = company_dim.replace(line_terminators, replacer, regex=True)
skills_dim = skills_dim.replace(line_terminators, replacer, regex=True)
skills_job_dim = skills_job_dim.replace(line_terminators, replacer, regex=True)

## Create CSV files

In [40]:
# Get the current working directory
current_dir = get_ipython().run_line_magic('pwd', '')

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Create a folder (csv_files) within the parent directory
csv_files_dir = os.path.join(parent_dir, 'csv_files')
os.makedirs(csv_files_dir, exist_ok=True)

# create a CSV file for each dataframe and store in the csv_files directory within the root directory ensure they are encoded as utf-8
# utf-8 encoding is needed as the data contains special characters
job_postings_fact.to_csv(os.path.join(csv_files_dir, 'job_postings_fact.csv'), index=False, encoding='utf-8')
company_dim.to_csv(os.path.join(csv_files_dir, 'company_dim.csv'), index=False, encoding='utf-8')
skills_dim.to_csv(os.path.join(csv_files_dir, 'skills_dim.csv'), index=False, encoding='utf-8')
skills_job_dim.to_csv(os.path.join(csv_files_dir, 'skills_job_dim.csv'), index=False, encoding='utf-8')

## Create .db file (SQLite)

In [41]:
# import sqlite3
# import subprocess

# # Connect to the database (or create it if it doesn't exist)
# conn = sqlite3.connect('jobs_2023.db')

# # Create the tables in the database using the four main dataframes
# job_postings_fact.to_sql('job_postings_fact', conn, if_exists='replace', index=False)
# company_dim.to_sql('company_dim', conn, if_exists='replace', index=False)
# skills_dim.to_sql('skills_dim', conn, if_exists='replace', index=False)
# skills_job_dim.to_sql('skills_job_dim', conn, if_exists='replace', index=False)

# # Close the connection
# conn.close()